In [ ]:
#! pyspark --packages org.mongodb.spark:mongo-spark-connector_2.10:2.2.1

### Load data from mongodb

In [1]:
data = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
            .option("uri","mongodb://34.210.118.215/gameofspark.train").load()

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### basic EDA

In [2]:
data.rdd.getNumPartitions()

250

In [3]:
data.printSchema()

root
 |-- C1: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- click: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- id: double (nullable = true)
 |-- site_category: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_id: 

In [4]:
data.show(5)

+----+-----+---+---+----+---+---+------+---+--------------------+------------+----------+--------+----------+-----+----------------+---------+---------+------------+-----------+--------+--------------------+-------------+-----------+--------+
|  C1|  C14|C15|C16| C17|C18|C19|   C20|C21|                 _id|app_category|app_domain|  app_id|banner_pos|click|device_conn_type|device_id|device_ip|device_model|device_type|    hour|                  id|site_category|site_domain| site_id|
+----+-----+---+---+----+---+---+------+---+--------------------+------------+----------+--------+----------+-----+----------------+---------+---------+------------+-----------+--------+--------------------+-------------+-----------+--------+
|1005|15706|320| 50|1722|  0| 35|100084| 79|[5a594db4e29072a6...|    07d7df22|  7801e8d9|ecad2386|         0|    0|               0| a99f214a| e8275b8f|    6332421a|          1|14102100|1.000064072448083...|     28905ebd|   f3845767|1fbe01fe|
|1005|18993|320| 50|2161|  0

In [3]:
data.groupBy(data['click']).count().show()

+-----+--------+
|click|   count|
+-----+--------+
|    1| 6865066|
|    0|33563901|
+-----+--------+



In [4]:
data.groupBy(data['hour']).count().show()

+--------+------+
|    hour| count|
+--------+------+
|14102412|188407|
|14102405|168185|
|14102514|242500|
|14102712|166229|
|14102804|140573|
|14102823|113551|
|14102323| 90232|
|14102913|193327|
|14102211|386757|
|14102501| 80637|
|14102121| 89501|
|14103013|250441|
|14102505|117207|
|14102821|166216|
|14102303|175094|
|14102608|188781|
|14102922| 90365|
|14103012|249022|
|14102311|161494|
|14102921| 95752|
+--------+------+
only showing top 20 rows



hour is in ***yymmddhh*** format <br>
**need to extract the last two digits, perhaps other timestamp info (eg. weekday/weekend)**

In [20]:
for c in data.columns:
    print c, ':\t', data.select(c).distinct().count()

C1 :	7
C14 :	2626
C15 :	8
C16 :	9
C17 :	435
C18 :	4
C19 :	68
C20 :	172
C21 :	60
_id :	40428967
app_category :	36
app_domain :	559
app_id :	8552
banner_pos :	7
click :	2
device_conn_type :	4
device_id :	2685699
device_ip :	6727799
device_model :	8251
device_type :	5
hour :	240
id :	40428967
site_category :	26
site_domain :	7745
site_id :	4736


In [10]:
data.groupBy(data['device_type']).count().show()

+-----------+--------+
|device_type|   count|
+-----------+--------+
|          1|37304667|
|          5|  129185|
|          4|  774272|
|          2|      31|
|          0| 2220812|
+-----------+--------+



### feature

In [21]:
import pyspark.sql.functions as F

In [33]:
# extract the last 2 digits of 'hour' column
data = data.withColumn('hh', (F.col('hour')%100))
# extract the day of week info from 'hour' column
data = data.withColumn('dow', (F.col('hour')/100-141019).cast('int')%7)
# extract the first 3 digits of ip address
data = data.withColumn('ip', 
                       F.conv(data.device_ip.substr(0,2),16,10).alias('deci').cast('int'))

In [35]:
data.select('hh').distinct().show()

+---+
| hh|
+---+
| 12|
| 22|
|  1|
| 13|
|  6|
| 16|
|  3|
| 20|
|  5|
| 19|
| 15|
|  9|
| 17|
|  4|
|  8|
| 23|
|  7|
| 10|
| 21|
| 11|
+---+
only showing top 20 rows



In [43]:
data.select('dow').distinct().show()

+---+
|dow|
+---+
|  1|
|  6|
|  3|
|  5|
|  4|
|  2|
|  0|
+---+



In [46]:
data.withColumn('ip', F.conv(data.device_ip.substr(0,2),16,10).alias('deci').cast('int'))\
    .select('ip').show(10)

+---+
| ip|
+---+
|232|
|150|
|  5|
|150|
|178|
| 55|
|230|
|221|
|179|
|241|
+---+
only showing top 10 rows



In [45]:
print 'MongoDB storage size:\t', 4962193408/1024/1024/1024.0
print 'csv file storage size:\t', 6311147778/1024/1024/1024.0
print 'ratio:\t', 4962193408/6311147778.0

MongoDB storage size:	4.62109375
csv file storage size:	5.876953125
ratio:	0.786258471921
